In [3]:
import numpy as np 
import itertools
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import random
from time import process_time

In [4]:
def rastrigin(x):
    return 10*len(x)+sum(x**2-10*np.cos(2*np.pi*x))

In [5]:
def sphere(x):
    return x.dot(x)

In [6]:
class Particle: # all the material that is relavant at the level of the individual particles
    
    def __init__(self, dim, minx, maxx, funct):
        self.position = np.random.uniform(low=minx, high=maxx, size=dim)
        self.velocity = np.random.uniform(low=-0.1, high=0.1, size=dim)
        self.best_particle_pos = self.position
        self.dim = dim
        self.funct=funct
        self.fitness = self.funct(self.position)
        self.best_particle_fitness = self.fitness   # we couldd start with very large number here, 
                                                    #but the actual value is better in case we are lucky 
                
    def setPos(self, pos):
        self.position = pos
        self.fitness = self.funct(pos)  #WATCH OUT WITH THIS LINE: IS IT self.position or just pos?
        
        if self.fitness<self.best_particle_fitness:     # to update the personal best both 
                                                        # position (for velocity update) and
                                                        # fitness (the new standard) are needed
                                                        # global best is update on swarm leven
            self.best_particle_fitness = self.fitness
            self.best_particle_pos = pos

    def updateVel(self, inertia, a1, a2, best_self_pos, best_swarm_pos):
                # Here we use the canonical version
                # V <- inertia*V + a1r1 (peronal_best - current_pos) + a2r2 (global_best - current_pos)
        cur_vel = self.velocity
        r1 = np.random.uniform(low=0, high=1, size = self.dim)
        r2 = np.random.uniform(low=0, high=1, size = self.dim)
        a1r1 = np.multiply(a1, r1)
        a2r2 = np.multiply(a2, r2)
        best_self_dif = np.subtract(best_self_pos, self.position)
        best_swarm_dif = np.subtract(best_swarm_pos, self.position)
                    # the next line is the main equation, namely the velocity update, 
                    # the velocities are added to the positions at swarm level
        new_vel = inertia*cur_vel + np.multiply(a1r1, best_self_dif) + np.multiply(a2r2, best_swarm_dif)
        self.velocity = new_vel
        return new_vel
    
    

#In these lines of codes, I add another argument to the particle class created in the labs, which determines the function in which our swarm particles are evaluated. 
#Hence, when optimising a particular function f, we will call our particles with funct=f.

In [7]:

class PSO: # all the material that is relavant at swarm leveel

    def __init__(self, w, a1, a2, dim, population_size, search_range, f, termination):

        # Here we use values that are (somewhat) known to be good
        # There are no "best" parameters (No Free Lunch), so try using different ones
        # There are several papers online which discuss various different tunings of a1 and a2
        # for different types of problems
        self.w = w # Inertia
        self.a1 = a1 # Attraction to personal best
        self.a2 = a2 # Attraction to global best
        self.dim = dim
        self.f=f
        self.termination=termination # a termination threeshold.
        self.swarm = [Particle(dim,-search_range,search_range, self.f) for i in range(population_size)]

        # Initialising global best, you can wait until the end of the first time step
        # but creating a random initial best and fitness which is very high will mean you
        # do not have to write an if statement for the one off case
        self.best_swarm_pos = np.random.uniform(low=-500, high=500, size=dim)
        self.best_swarm_fitness = 1e100

    def run(self): 
        time=0
        diver=0
        while self.best_swarm_fitness>self.termination:
            time=time+1
            for p in range(len(self.swarm)):
                particle = self.swarm[p]

                new_position = particle.position + particle.updateVel(self.w, self.a1, self.a2, particle.best_particle_pos, self.best_swarm_pos)
                                
                if new_position.dot(new_position) > 1.0e+18: # The search will be terminated if the distance 
                                                        # of any particle from center is too large
                        # We would want parameters for which our swarm does not diverge. As we also do not want 
                        # parameter settings producing time consuming runs, we group these two options by making divergent swarms look as time consuming runs
                        #print('Time:', time,'Best Pos:',self.best_swarm_pos,'Best Fit:',self.best_swarm_fitness)
                        #This will tell us whether the swarm diverged or not.
                        #'Most likely divergent: Decrease parameter values'
                    diver=1
                    return [None, diver] #if our new particle position is too far apart, we simply return a None time value and a 1 divergence value.
                        #This will state that most likely our swarm diverged and consequently there is no proper convergence of the swarm.
                
                if time==1000: 
                    return [None,1]# To avoid to much time running.
                
                self.swarm[p].setPos(new_position)         #When executing this recall that you are also maybe updating the value of the particle's best.

                new_fitness = self.f(new_position)

                if new_fitness < self.best_swarm_fitness:   # to update the global best both 
                                                            # position (for velocity update) and
                                                            # fitness (the new group norm) are needed
                    self.best_swarm_fitness = new_fitness
                    self.best_swarm_pos = new_position
                
        return [time, diver]
                
# review time and diver variables.


In [6]:
PSO(0.3, 1, 1, 4, 50, 5.12, sphere, 0.00001).run()

[13, 0]

In [11]:
class DifferentialEvolution:
    def __init__(self,dim, F, prob, size, minx, maxx, funct):
        self.dim=dim
        self.F=F
        self.prob=prob
        self.size=size
        self.minx=minx
        self.maxx=maxx
        self.funct=funct
        self.population=[np.random.uniform(low=self.minx, high=self.maxx, size=self.dim) for n in range(self.size)]
    
    def newpopulation(self):
        for n in range(self.size):
            individual=self.population[n]
            v=np.zeros(self.dim)
            three_parents=random.sample(self.population,3)
#             print(three_parents)
            if self.funct(three_parents[0])>self.funct(three_parents[1]):
                copy_individual=individual
                v=three_parents[2]+self.F*(three_parents[1]-three_parents[0])
                for dimension in range(self.dim):
                    if np.random.uniform()<self.prob:
                        copy_individual[dimension]=v[dimension]
                        
                if self.funct(copy_individual)<self.funct(individual):
                    print(self.funct(individual))
                    print(self.funct(copy_individual))
                    self.population[n]=copy_individual
                    
    def run(self, term_criterion, max_iterations):
        time=0
        while (min(np.apply_along_axis(self.funct, 0, self.population))>term_criterion):
            if time>=max_iterations:
                return [min(np.apply_along_axis(self.funct, 0, self.population)),time,1]
            time=time+1
            self.newpopulation()
            for individual in self.population:
                if individual.dot(individual)> 1.0e+18:
                    return [min(np.apply_along_axis(self.funct, 0, self.population)),time,1]
        return min(np.apply_along_axis(self.funct, 0, self.population)), time, 0

In [28]:
t_start=process_time()
print(DifferentialEvolution(4,np.sqrt((1-(p/2))/N),p, 40,-5.12,5.12,rastrigin).run(0.00001,1000))
t_stop=process_time()
print(t_stop-t_start)

[30.70166829497135, None, 1]
0.390625


In [16]:
t_start=process_time()
print(PSO(0.7, 2, 2, 4, 24, 5.12, rastrigin, 0.00001).run())
t_stop=process_time()
print(t_stop-t_start)

[478, 0]
0.375


In [14]:
range_p=[0,0.25,0.5,0.75,1]
x=list()
undesired=list()
seconds=list()
times=list()
end_posis=list()
for p in range_p: 
    c=0
    sumatimes=0
    sumadiver=0
    sumaseconds=0
    sumaposi=0
    for it in range(300): #the average time we take will be taken over 300 sets
        
            t_start=process_time() #for timing in seconds
            results=DifferentialEvolution(4,np.sqrt((1-(p/2))/40),p, 40,-5.12,5.12,rastrigin).run(0.00001,1000)
            t_stop=process_time()
            posi,time,diver=results
            sumadiver=sumadiver+1 
            sumatimes=sumatimes+time
            sumaseconds=sumaseconds+t_stop-t_start
            sumaposi=sumaposi+posi
            c=c+1
                
    averagetime=sumatimes/c 
    averagediver=sumadiver/c
    averageseconds=sumaseconds/c
    averageposi=sumaposi/c
    
    times.append(averagetime)
    undesired.append(averagediver)
    seconds.append(averageseconds)
    end_posis.append(averageposi)

In [16]:
print(times)
print(seconds)
print(undesired)
print(end_posis)

[1000.0, 1000.0, 1000.0, 1000.0, 1000.0]
[1.375625, 1.3884375, 1.3553125, 1.335, 1.4653125]
[1.0, 1.0, 1.0, 1.0, 1.0]
[676.8959669017688, 214.61654139680482, 65.44030584218713, 57.066285372100126, 132.8868497461566]
